In [7]:
import re
str1 = '[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}'
re.findall('\{(.*?)\}', str1)

['timestamp', '', 'txAtt:d', '', 'torTemperature:d', '', 'avgIMpa0:d', '']

In [ ]:
import json
import uuid
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}
for index in range(12):
    item = {}
    item['role_path'] = 'root.' + 'branch' + str(branch[index]) + '.search_branch' + str(branch[index])
    item['identifier'] = 'search_branch' + str(branch[index])
    item['desc'] = 'Branch ' + str(branch[index]) + ' common use KeyValue and Mark'
    item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+').*(txAtt|Pma|linearization fault)'
    item['exp_extract'] = [
                        "{}[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "{}[{timestamp}] {}"
                        ]
    item['exp_mark'] = [{"abbr":"LF","exp":"linearization fault","color":"#f00000"}]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    if index == 0:
        item['is_active'] = True
    config['search'].append(item)

keys = ['txAtt', 'torTemperature', 'avgIMpa0', 'LF']
for search_config in config['search']:
    item = {}
    item['role_path'] = search_config['role_path'].replace('search', 'chart')
    item['identifier'] = search_config['identifier'].replace('search', 'chart')
    item['desc'] = item['identifier'] + ' all KeyValue and Mark'
    item['check'] = False
    item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
    for search_config2 in config['search']:
        search_atom = {}
        search_atom['namespace'] = search_config2['identifier']
        search_atom['name'] = search_config2['identifier']
        search_atom['check'] = False
        search_atom['children'] = []
        for key in keys:
            if search_config['identifier'] == search_config2['identifier']:
                search_atom['children'].append({'name': key, 'check': True})
            else:
                search_atom['children'].append({'name': key, 'check': False})
        item['key_value_tree']['children'].append(search_atom)
    config['chart'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'insight_branch' + str(branch[index])
#     item['desc'] = 'Branch Insight' + str(branch[index])
#     item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+')'
#     item['exp_extract'] = "{}[{timestamp}]{}, msg = {msg}"
#     item['exp_mark'] = {"name":"LF","exp":"linearization fault","color":"#f00000"}
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['insight'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'statistic_branch' + str(branch[index])
#     item['desc'] = 'Branch ' + str(branch[index]) + ' statistic txAtt'
#     item['code'] = "self.result = self.text_file_model.data['" + 'search_branch' + str(branch[index]) + "'].res_key_value['txAtt']['global_index']"
#     config['statistic'].append(item)
    
json_object = json.dumps(config)
with open("E:\\projects\\ericsson_flow\\new_files\\config.ecfg", "w") as outfile:
    outfile.write(json_object)

In [10]:
import json
import uuid
import re

def add_search_config(parent, express, processes, keyword, mark):
    config = []
    
    item = {}
    item['identifier'] = f'{parent}_ErrorFaultABN'
    identifier = item['identifier']
    item['role_path'] = f'root.{parent}.EFA.{identifier}'

    item['desc'] = parent + ' Error Fault ABN'
    item['exp_search'] = parent + '.*(error| fault|ABN)'
    item['exp_extract'] = [
                            parent + express
                        ]
    item['exp_mark'] = [
                            {"abbr":"ER","exp":"error","color":"#f00000"},
                            {"abbr":"FA","exp":" fault","color":"#f00000"},
                            {"abbr":"ABN","exp":"ABN","color":"#ffff00"}
                        ]
    item['is_active'] = True
    item['is_case_sensitive'] = False
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config.append(item)

    for index, process in enumerate(processes):
        item = {}
        name = process.replace(',','_')
        item['identifier'] = f'{parent}_{name}'
        identifier = item['identifier']
        item['role_path'] = f'root.{parent}.{identifier}' if (keyword not in process) else f'root.{parent}.{process}.{identifier}'
        item['desc'] =  parent + ' ' + process + ' common use KeyValue and Mark'
        item['exp_search'] = f'{parent}.*{process}'
        item['exp_extract'] = [
                                parent + express
                            ]
        item['exp_mark'] = mark if (keyword in process) else []
        item['is_active'] = False
        item['is_case_sensitive'] = True
        item['forward_rows'] = 0
        item['backward_rows'] = 0
        config.append(item)
    return config
        
def add_chart_config(tmp_search):
    config = []
    for search_config in tmp_search:

        if len(search_config['exp_mark']) == 0:
            continue

        item = {}
        item['role_path'] = search_config['role_path'] + '_Chart'
        item['identifier'] = search_config['identifier'] + '_Chart'
        item['desc'] = item['identifier'] + ' all KeyValue and Mark'
        item['check'] = False
        item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
        for search_config2 in tmp_search:
            search_atom = {}
            search_atom['namespace'] = search_config2['identifier']
            search_atom['name'] = search_config2['identifier']
            search_atom['check'] = False
            search_atom['children'] = []
            if len(search_config2['exp_mark']) > 0:
                keys = []
                for mark in search_config2['exp_mark']:
                    keys.append(mark['abbr'])
                    
                for key in keys:
                    if search_config['identifier'] == search_config2['identifier']:
                        search_atom['children'].append({'name': key, 'check': True})
                    else:
                        search_atom['children'].append({'name': key, 'check': False})
                        
            item['key_value_tree']['children'].append(search_atom)
        config.append(item)
    return config

config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\DOT TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(' \((.*?)\): ', line)
        if len(res) > 0:
            processes.append(res[0]) 
processes = list(set(processes))
processes.sort()
config['search'].extend(add_search_config('DOT_TE_LOG', ":{timestamp} ({}", processes, 'kkkkkkkkkkkkk', []))

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\DU1 TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall('procname = \"(.*?)\"', line)
        if len(res) > 0:
            processes.append(res[0])
processes = list(set(processes))
processes.sort()
config['search'].extend(add_search_config('DU1_TE_LOG', ":[{timestamp}]{}", processes, 'kkkkkkkkkkkkk', []))
config['search'].extend(add_search_config('DU2_TE_LOG', ":[{timestamp}]{}", processes, 'kkkkkkkkkkkkk', []))

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\RU1 TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(', { \"(.*?)\"', line)
        if len(res) > 0:
            processes.append(res[0])
processes = list(set(processes))
processes.sort()
mark = [
            {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#759aa0"},
            {"abbr":"ER","exp":"EVENT_RELEASE","color":"#e69d87"},
            {"abbr":"ES","exp":"EVENT_SETUP","color":"#8dc1a9"},
            {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#ea7e53"},
            {"abbr":"IND","exp":"IND_ACTIVATE","color":"#eedd78"}
        ]
tmp_search = add_search_config('RU1_TE_LOG', '{}[{timestamp}]{}', processes, 'TxBranchCtrl', mark)
config['search'].extend(tmp_search)
config['chart'].extend(add_chart_config(tmp_search))

tmp_search = add_search_config('RU2_TE_LOG', '{}[{timestamp}]{}', processes, 'TxBranchCtrl', mark)
config['search'].extend(tmp_search)
config['chart'].extend(add_chart_config(tmp_search))

json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\DotConfigFull.ecfg", "w") as outfile:
    outfile.write(json_object)

In [3]:
import re
from parse import parse

str1 = 'RU2_TE_LOG:BXP_2: [2023-02-23 10:31:25.766459124] com_ericsson_trithread:INFO: { 0 }, { "TxBranchCtrlC", "logManager.cc:37", "Branch does not support different power level (txChangeCycleStatePrepare.cc:517)" }'
# str2 = 'BXP_3: [2022-12-10 15:55:26.739019220] (+0.000025340) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "TxBranchCtrlB", fileAndLine = "txChangeCycleHelper.cc:264", msg = "Txl branch J restart due to txL linearization fault!" }'
str1 = "	fru_2051: [230201 120650]  144: PSU enters: DC_AISG_U:0 exceeds the limists. Value(mV): 587, limits[EXCEPTIONAL_HI,NORMAL_HI, NORMAL_LOW, EXCEPTIONAL_LOW](13125,13125,11875,11875)"
# exp = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# exp = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
exp_extract = '{}[{timestamp}]{}PSU enters{}'
r_extract = parse(exp_extract, str1)
print(r_extract)

<Result ('\tfru_2051: ', '  144: ', ': DC_AISG_U:0 exceeds the limists. Value(mV): 587, limits[EXCEPTIONAL_HI,NORMAL_HI, NORMAL_LOW, EXCEPTIONAL_LOW](13125,13125,11875,11875)') {'timestamp': '230201 120650'}>


In [4]:
import re

# processes = []
# with open("D:\\projects\\ericsson_flow\\new_files\\4449\\faillog 4449 ULSA.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall("{ \"(.*?)\",", line)
#         if len(res) > 0:
#             # if ('BranchCtrl' not in res[0]) & ('PhaseCtrl' not in res[0]):
#             processes.append(res[0])
            
# with open("D:\\projects\\ericsson_flow\\new_files\\4449\\passlog 4449 ULSA.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall("{ \"(.*?)\",", line)
#         if len(res) > 0:
#             # if ('BranchCtrl' not in res[0]) & ('PhaseCtrl' not in res[0]):
#             processes.append(res[0])
            
# processes = list(set(processes))
# processes.sort()
# processes
processes = [
 'AntennaModule',
 'EquipCtrl',
 'PaSrvDC',
 'PimcTdiHandler',
 'RICRAI_SERVER',
 'RxBranchCtrlA',
 'RxBranchCtrlB',
 'RxBranchCtrlC',
 'RxBranchCtrlD',
 'RxRmcService',
 'TorLoSrvCD',
 'TxBranchCtrlC',
 'TxBranchCtrlD',
 'TxCoordinationSrvCD',
 'TxLoSrvC',
 'TxLoSrvD',
 'TxTimingPhaseCtrlC',
 'antpServerProc_',
 'bcProc',
 'brhProc',
 'carrierListElogScheduler.cc',
 'carrierResourceHandlerImpl.cc',
 'child.c',
 'cmd_proc',
 'cmd_proc(COLI)',
 'cpriCtrlProc',
 'cprilhmd.c',
 'crhCarrierCtrl.cc',
 'crhCarrierListHandler.cc',
 'crhProxyProc',
 'ecp_x11.c',
 'faultManagerProc',
 'iqIqcSwitchingAlgorithmCpri.cc',
 'iqIqcSwitchingResourceHandlerIf.cc',
 'iqIqcSwitchingResourceHandlerMs.cc',
 'ledProc',
 'libecp_x11.c',
 'lrciRasCapDbFacadeAbfaas.cc',
 'lrciRasCapDbFacadeBase.cc',
 'lrciRasCapabilities.cc',
 'radioTrDcServer',
 'rfPort.cc',
 'rftProc',
 'rhd-lh.c',
 'rmcProc',
 'rtsCtrl',
 'rtsCtrl_requestProcessor',
 'rxBranchTimingInfoReporter.cc',
 'stimenotifyd.c',
 'tmoSchedulerEqp',
 'tmoSchedulerTx',
 'trDcProc',
 'txBranchReporterCrh.cc',
 'txlProcBranchC',
 'txlProcBranchD',
 'ulh_hdlc.c']

In [7]:
import json
import uuid

product_name = 'Radio4449'
processes = processes
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

item = {}
item['role_path'] = f'{product_name}.TimedoutErrorFaultABN'
item['identifier'] = 'TimedoutErrorFaultABN'
item['desc'] = 'Error Fault ABN Timedout'
item['exp_search'] = 'error| fault|ABN|Timed out'
item['exp_extract'] = [
                    "[{timestamp}] {}" 
                    ]
item['exp_mark'] = [
                        {"abbr":"TO","exp":"Timed out","color":"#FFB6C1"},
                        {"abbr":"ER","exp":"error","color":"#f00000"},
                        {"abbr":"LF","exp":"linearization fault","color":"#f00000"},
                        {"abbr":"ABN","exp":"ABN:","color":"#ffff00"}
                    ]
item['is_active'] = True
item['is_case_sensitive'] = True
item['forward_rows'] = 0
item['backward_rows'] = 0
config['search'].append(item)

for index, process in enumerate(processes):
    item = {}
    item['role_path'] = f'{product_name}.' + process.replace('.','_').replace('(','_').replace(')','_')
    item['identifier'] = process
    item['desc'] =  process + ' common use KeyValue and Mark'
    item['exp_search'] = '"'+process.replace('(','\\(').replace(')','\\)')+'", '
    item['exp_extract'] = [
                        # "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "[{timestamp}] {}" 
                        ]
    # item['exp_mark'] = [
    #                         {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#e00000"},
    #                         {"abbr":"ER","exp":"EVENT_RELEASE","color":"#d00000"},
    #                         {"abbr":"ES","exp":"EVENT_SETUP","color":"#c00000"},
    #                         {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#b00000"},
    #                         {"abbr":"IND","exp":"IND_ACTIVATE","color":"#a00000"}
    #                     ]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config['search'].append(item)

# tmp_search = []
# processes = range(2)
# for index in processes:
#     item = {}
#     branch_num = str(index)
#     item['role_path'] = f'{product_name}.' + 'branch' + branch_num + '.search_branch' + branch_num
#     item['identifier'] = 'search_branch' + branch_num
#     item['desc'] = 'Branch ' + branch_num + ' common use KeyValue and Mark'
#     item['exp_search'] = f'(process = "txlProcBranch'+branch_num+'"|process = "TxBranchCtrl'+branch_num+'")'
#     item['exp_extract'] = [
#                         "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
#                         # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
#                         "[{timestamp}] {}" 
#                         ]
#     item['exp_mark'] = [
#                             {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#759aa0"},
#                             {"abbr":"ER","exp":"EVENT_RELEASE","color":"#e69d87"},
#                             {"abbr":"ES","exp":"EVENT_SETUP","color":"#8dc1a9"},
#                             {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#ea7e53"},
#                             {"abbr":"IND","exp":"IND_ACTIVATE","color":"#eedd78"}
#                         ]
#     item['is_active'] = False
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['search'].append(item)
#     tmp_search.append(item)
        
# keys = ['txAtt', 'avgIMpa0', 'ED', 'ER', 'ES', 'EA', 'IND']
# for search_config in tmp_search:
#     item = {}
#     item['role_path'] = search_config['role_path'].replace('search', 'chart')
#     item['identifier'] = search_config['identifier'].replace('search', 'chart')
#     item['desc'] = item['identifier'] + ' all KeyValue and Mark'
#     tree = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
#     for search_config2 in tmp_search:
#         search_atom = {}
#         search_atom['namespace'] = search_config2['identifier']
#         search_atom['name'] = search_config2['identifier']
#         search_atom['check'] = False
#         search_atom['children'] = []
#         for key in keys:
#             if search_config['identifier'] == search_config2['identifier']:
#                 search_atom['children'].append({'name': key, 'check': True})
#             else:
#                 search_atom['children'].append({'name': key, 'check': False})
#         tree['children'].append(search_atom)
#     item['key_value_tree'] = [{'id': '', 'type': 'TidyTree', 'elements':tree}]
#     config['chart'].append(item)
    
json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\4449Config.ecfg", "w") as outfile:
    outfile.write(json_object)

In [ ]:
import zipfile
import gzip
import io
from utils import *

class ZipDecompress(object):
    def __init__(self):
        self.path = path
        self.files = {}
        
    def decompress(self, extract_paths):
        with zipfile.ZipFile(self.path, 'r') as file:
            for extract_path in extract_paths:
                file_name, lines = self.decompress_file(file, extract_path)
                self.files[file_name] = lines
        
    def decompress_file(self, f_refer, extract_path):
        path_node = extract_path.split('/')[0].replace('*', '')
        tg = ''
        for file in f_refer.filelist:
            if path_node in file.filename:
                tg = file.filename

        if '.zip' in tg:
            with f_refer.open(tg, 'r') as nest:
                zfile = io.BytesIO(nest.read())
                with zipfile.ZipFile(zfile) as nested_zip:
                    return extract_file(nested_zip, '/'.join(extract_path.split('/')[1:]))
        elif '.gz' in tg:
            with f_refer.open(tg, 'r') as nest:
                zfile = io.BytesIO(nest.read())
                lines = gzip.open(zfile, 'r')
                return nest.name, [line.decode("utf-8") for line in lines]
        else:
            with f_refer.open(tg, 'r') as nest:
                lines = nest.readlines()
                return tg, [line.decode("utf-8") for line in lines]
            
class FileSearch(object):
    def __init__(self, lines):
        self.lines = lines
        
        
    def search_by_range(self, start_exp, end_exp):
        res = []
        
        flag = False
        lines = []
        assigned_name = ''
        for index, line in enumerate(self.lines):
            if flag:
                r = parse(end_exp, line)
                if r is not None:
                    flag = False
                    res.append(lines)
                    lines = []
                else:
                    lines.append({'global_index':index, 'text':line})
                    continue
            r = parse(start_exp, line)
            if r is not None:
                for key in r.named.keys():
                    
                if r['assigned_name'] not in files_cmds_content[file_name][cmd]:
                    files_cmds_content[file_name][cmd][r['assigned_name']] = []
                flag = True
                assigned_name = r['assigned_name']
                lines.append({'global_index':index, 'text':line})
                    
    def search_by_regular(self):
        pass
            
class TextExtract(object):
    def __init__(self, lines):
        self.lines = lines

    # init 
    files_cmds_content = {}
    for file_name in configs.keys():
        files_cmds_content[file_name] = {}
        for cmd in configs[file_name]['cmds'].keys():
            files_cmds_content[file_name][cmd] = {}

    # find cmd content
    for file_name in files_origin_lines.keys():
        for cmd in configs[file_name]['cmds'].keys():
            flag = False
            lines = []
            assigned_name = ''
            for index, line in enumerate(files_origin_lines[file_name]):
                if flag:
                    r = parse(cmd['end_exp'], line)
                    if r is not None:
                        flag = False
                        files_cmds_content[file_name][cmd][assigned_name].append(lines)
                        lines = []
                    else:
                        lines.append({'global_index':index, 'text':line})
                        continue
                r = parse(cmd['start_exp'], line)
                if r is not None:
                    if r['assigned_name'] not in files_cmds_content[file_name][cmd]:
                        files_cmds_content[file_name][cmd][r['assigned_name']] = []
                    flag = True
                    assigned_name = r['assigned_name']
                    lines.append({'global_index':index, 'text':line})

    # extract key value
    for file_name in files_origin_lines.keys():
        for cmd in configs[file_name]['cmds'].keys():
            for assigned in files_cmds_content[file_name][cmd].keys():
                cmd_batch = files_cmds_content[file_name][cmd][assigned]
                for batch_index, lines in enumerate(cmd_batch):
                    tmp = {}
                    cmd_config = configs[file_name]['cmds'][cmd]
                    for extract_key in cmd_config['extract_exps'].keys():
                        if extract_key != 'timestamp':
                            tmp[extract_key] = []

                    if cmd_config['time_type'] == 'embedded':
                        for line in lines:
                            for extract_key in cmd_config['extract_exps'].keys():
                                r = parse(cmd_config['extract_exps'][extract_key]['exp'], line['text'])
                                if r is not None:
                                    item = {
                                            'file_name':file_name, 'assigned_name':content['assigned_name'], 
                                            'cmd':cmd, 'keywords':extract_key, 'global_index': line['global_index'], 'value':'',
                                            'cmd_batch_index': batch_index, 'timestamp': convert_datetime_timestamp(r.named['timestamp']), 'x': random.randint(0, 100), 'y':random.randint(0, 100)
                                            }
                                    tmp[extract_key].append(item)
                    elif exps[key]['time_type'] == 'batch':
                        timestamp = ''
                        for line in lines:
                            for extract_key in cmd_config['extract_exps'].keys():
                                r = parse(cmd_config['extract_exps'][extract_key]['exp'], line['text'])
                                if r is not None:
                                    if 'timestamp' in r.named:
                                        timestamp = convert_datetime_timestamp(r.named['timestamp'])
                                        break
                                    item = {
                                            'file_name':file_name, 'product_name':table[content['assigned_name']], 'assigned_name':content['assigned_name'], 
                                            'cmd':key, 'keywords':extract_key, 'global_index': line['global_index'], 'value':r.named[extract_key],
                                            'cmd_batch': cmd_batch, 'timestamp': timestamp, 'x': random.randint(0, 100), 'y':random.randint(0, 100)
                                            }
                                    tmp[extract_key].append(item)

                    for keyword in tmp.keys():
                        max_len = max([len(v) for v in tmp.values()])
                        for k, v in tmp.items():
                            if len(v) < max_len:
                                v.extend([None] * (max_len - len(v)))

                        an = keyword+'_abnormal'
                        ptmp = pd.DataFrame(tmp)
                        if len(ptmp) == 0:
                            break
                        ptmp = ptmp.reset_index().rename(columns={'index':'occur_batch'})
                        ptmp = ptmp.apply(lambda x: abnormal_condition(x, exps[key]['extract_exps'][keyword]['cond']), axis=1)
                        abnormal = []
                        for item,occur_batch  in ptmp.loc[(ptmp[an] == True), [keyword, 'occur_batch']].values:
                            item['occur_batch'] = occur_batch
                            abnormal.append(item)
                        res.append({'file_name': file_name, 'product_name': table[content['assigned_name']],'assigned_name': content['assigned_name'], 'cmd': key, 'keywords': keyword, 'abnormal':abnormal})

In [ ]:
dcgm = 'D:/projects/ericsson_flow/new_files/Dot4455/AKH167_242367_230215_112831_CET_MSRBS-LN_CXP2010174-1_R56J19_dcgm.zip'
configs = {
    'telog':{'extract_path': '*_logfiles.zip/teread.log', 'cmds':{
                'telog read':{'extract_exps': [], 
                             'time_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
                            }},
    # 'elog':{'extract_path': '*_e2.log.gz/*_dcg_e2.log', 'cmds':[
    #             {'extract_exps': [], 
    #              'cmd':'', 'time_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
    #             ]},
    # 'modump':{'extract_path': '*_modump.zip/*_dcg_k.log.gz/*_dcg_k.log', 'cmds':[
    #             {'extract_exps': [], 
    #              'cmd':'', 'time_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
    #             ]},
}